In [2]:
import os
import gc
import sys
import time
import json
import glob
import random
from pathlib import Path
import pandas as pd

from PIL import Image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa

import itertools
from tqdm import tqdm

import warnings 
warnings.filterwarnings("ignore")

## Load raw data

In [34]:
training_folder = "train/"
df = pd.read_csv("train_scaled_rev.01.csv")
df = df[df['class_id']!=14]
df

,image_id,dim0,dim1,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,x_dim,y_dim,x_min_scaled,x_max_scaled,y_min_scaled,y_max_scaled
9,7ecd6f67f649f26c05805c8359f9e528,2952,2744,Pleural thickening,11,R9,1769.0,396.0,2071.0,551.0,476,512,330.075802,386.425656,68.682927,95.566396
10,7ecd6f67f649f26c05805c8359f9e528,2952,2744,Pulmonary fibrosis,13,R10,748.0,557.0,1099.0,911.0,476,512,139.568513,205.061224,96.607046,158.005420
11,7ecd6f67f649f26c05805c8359f9e528,2952,2744,Pulmonary fibrosis,13,R9,714.0,597.0,1014.0,830.0,476,512,133.224490,189.201166,103.544715,143.956640
12,7ecd6f67f649f26c05805c8359f9e528,2952,2744,Pulmonary fibrosis,13,R8,810.0,612.0,1170.0,958.0,476,512,151.137026,218.309038,106.146341,166.157182
13,2229148faa205e881cf0d932755c9e40,2880,2304,Pulmonary fibrosis,13,R9,523.0,402.0,880.0,748.0,410,512,116.222222,195.555556,71.466667,132.977778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67906,e7ea94b2fec95a7461b10a7f3eea2897,3072,3072,Cardiomegaly,3,R9,1368.0,1279.0,2480.0,1737.0,512,512,228.000000,413.333333,213.166667,289.500000
67907,e7ea94b2fec95a7461b10a7f3eea2897,3072,3072,Aortic enlargement,0,R8,1747.0,529.0,2113.0,948.0,512,512,291.166667,352.166667,88.166667,158.000000
67908,e7ea94b2fec95a7461b10a7f3eea2897,3072,3072,Cardiomegaly,3,R10,1380.0,1448.0,2471.0,1710.0,512,512,230.000000,411.833333,241.333333,285.000000
67909,e7ea94b2fec95a7461b10a7f3eea2897,3072,3072,Aortic enlargement,0,R9,1703.0,494.0,2085.0,836.0,512,512,283.833333,347.500000,82.333333,139.333333


In [10]:
unique_count = df.groupby(["class_id","rad_id"]).image_id.count()
unique = df[["class_id","rad_id"]].drop_duplicates().values
index = []
for i in unique:
    count = unique_count[i[0]][i[1]]
    temp_df = df[df["class_id"]==i[0]]
    temp_df = temp_df[temp_df["rad_id"]==i[1]]
    temp_index = list(temp_df.index)
    if count < 150:
        index += temp_index
    else:
        index += temp_index[:150]
df = df.loc[index]

In [11]:
df

,image_id,dim0,dim1,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,x_dim,y_dim,x_min_scaled,x_max_scaled,y_min_scaled,y_max_scaled
9,7ecd6f67f649f26c05805c8359f9e528,2952,2744,Pleural thickening,11,R9,1769.0,396.0,2071.0,551.0,476,512,330.075802,386.425656,68.682927,95.566396
15,2229148faa205e881cf0d932755c9e40,2880,2304,Pleural thickening,11,R9,270.0,550.0,525.0,963.0,410,512,60.000000,116.666667,97.777778,171.200000
22,2229148faa205e881cf0d932755c9e40,2880,2304,Pleural thickening,11,R9,1479.0,395.0,1815.0,605.0,410,512,328.666667,403.333333,70.222222,107.555556
50,98617a2bbd11c4afa7be664889cdd6de,2861,2619,Pleural thickening,11,R9,1510.0,352.0,2119.0,675.0,469,512,295.196640,414.252768,62.993359,120.796924
52,98617a2bbd11c4afa7be664889cdd6de,2861,2619,Pleural thickening,11,R9,655.0,368.0,1178.0,708.0,469,512,128.048874,230.292478,65.856693,126.702552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52328,a673c55470f775d286249e189896c376,2308,2240,Pneumothorax,12,R14,1151.0,215.0,1956.0,960.0,497,512,263.085714,447.085714,47.694974,212.963605
52331,a673c55470f775d286249e189896c376,2308,2240,Pneumothorax,12,R11,1413.0,297.0,1690.0,526.0,497,512,322.971429,386.285714,65.885615,116.686308
58237,c5aa9d492605510ce5fc2403df57cd6a,2433,2298,Infiltration,6,R17,302.0,796.0,521.0,974.0,484,512,67.286336,116.080070,167.510070,204.968352
67895,dcb081bb5e1dac41000e96fc37c8c322,2651,2507,Consolidation,4,R14,212.0,352.0,1103.0,2248.0,484,512,43.296370,225.263662,67.983402,434.166730


## Function on image resize

In [12]:
def get_mask(img_dimensions, x_min, y_min, x_max, y_max):
    img_height, img_width = img_dimensions
    img_mask = np.full((img_height,img_width),0)
    img_mask[y_min:y_max,x_min:x_max] = 255
    
    return img_mask.astype(np.float32)


def rle_encoding(x):
    dots = np.where(x.T.flatten() == 255)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join([str(x) for x in run_lengths])

## Resize image and bonunding box in 512x512 as some image is smaller than 512x512

In [35]:
resized_folder = "resized_train/"
#os.mkdir(resized_folder)
diagnostic_per_image = []
pics = df['image_id'].unique()

image_size=512
with tqdm(total=len(df[['image_id','rad_id']].drop_duplicates())) as pbar:
    for pic in pics:
        image_id = pic
        image_df = df.query("image_id==@image_id")
        rad_ids = image_df['rad_id'].unique()
        for rad_id in rad_ids:
            image_rad_df = image_df.query('rad_id==@rad_id')
            class_list = []
            RLE_list = []
            for diagnostic_id, diagnostic in image_rad_df.iterrows():
                class_list.append(diagnostic.class_id)

                dicom_image = cv2.imread(training_folder+image_id+".png")
                image_dimensions = dicom_image.shape

                resized_img = cv2.resize(dicom_image, (image_size,image_size), interpolation = cv2.INTER_AREA)
                cv2.imwrite(resized_folder+image_id+".png", resized_img) 

                mask = get_mask([512,512], int(diagnostic.x_min_scaled), int(diagnostic.y_min_scaled), int(diagnostic.x_max_scaled), int(diagnostic.y_max_scaled))
                #resized_mask = cv2.resize(mask, (image_size,image_size))
                #RLE_list.append(rle_encoding(resized_mask))
                RLE_list.append(rle_encoding(mask))
            diagnostic_per_image.append({"image_id":image_id,
                                         "CategoryId":class_list,
                                         "EncodedPixels":RLE_list,
                                         "Rad_id":rad_id})
            pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 13182/13182 [14:06<00:00, 15.57it/s]


In [36]:
samples_df = pd.DataFrame(diagnostic_per_image)
samples_df["Height"] = image_size
samples_df["Width"] = image_size
samples_df.to_csv('train_512.csv')

In [37]:
samples_df

,image_id,CategoryId,EncodedPixels,Rad_id,Height,Width
0,7ecd6f67f649f26c05805c8359f9e528,"[11, 13]",[169029 27 169541 27 170053 27 170565 27 17107...,R9,512,512
1,7ecd6f67f649f26c05805c8359f9e528,[13],[71265 62 71777 62 72289 62 72801 62 73313 62 ...,R10,512,512
2,7ecd6f67f649f26c05805c8359f9e528,[13],[77419 60 77931 60 78443 60 78955 60 79467 60 ...,R8,512,512
3,2229148faa205e881cf0d932755c9e40,"[13, 11, 7, 11]",[59464 61 59976 61 60488 61 61000 61 61512 61 ...,R9,512,512
4,2229148faa205e881cf0d932755c9e40,"[11, 13, 13, 13, 11]",[34887 99 35399 99 35911 99 36423 99 36935 99 ...,R8,512,512
...,...,...,...,...,...,...
13177,c619a784636c085eb798f98a5ba1102d,"[9, 0]",[41733 64 42245 64 42757 64 43269 64 43781 64 ...,R9,512,512
13178,c619a784636c085eb798f98a5ba1102d,"[9, 8, 0]",[43276 46 43788 46 44300 46 44812 46 45324 46 ...,R10,512,512
13179,e7ea94b2fec95a7461b10a7f3eea2897,"[0, 3]",[147546 57 148058 57 148570 57 149082 57 14959...,R10,512,512
13180,e7ea94b2fec95a7461b10a7f3eea2897,"[3, 0]",[116950 76 117462 76 117974 76 118486 76 11899...,R9,512,512


In [38]:
dicom_image = cv2.imread(training_folder+"dcb081bb5e1dac41000e96fc37c8c322"+".png")
np.where(dicom_image.T.flatten() == 255)

(array([ 61957,  61958,  61959,  61960,  61961,  61962,  61963,  61964,
         61965,  61966,  61967,  61968,  61970,  64012,  64524,  64525,
         65037,  65038,  65541,  65546,  65551,  66054,  66057,  66064,
         66065,  66568,  66578, 236576, 237599, 238622, 238637, 240157,
        240682, 241194, 243741, 244253, 244765, 309765, 309766, 309767,
        309768, 309769, 309770, 309771, 309772, 309773, 309774, 309775,
        309776, 309778, 311820, 312332, 312333, 312845, 312846, 313349,
        313354, 313359, 313862, 313865, 313872, 313873, 314376, 314386,
        484384, 485407, 486430, 486445, 487965, 488490, 489002, 491549,
        492061, 492573, 557573, 557574, 557575, 557576, 557577, 557578,
        557579, 557580, 557581, 557582, 557583, 557584, 557586, 559628,
        560140, 560141, 560653, 560654, 561157, 561162, 561167, 561670,
        561673, 561680, 561681, 562184, 562194, 732192, 733215, 734238,
        734253, 735773, 736298, 736810, 739357, 739869, 740381],

In [39]:
dicom_image.T

array([[[ 62,  66,  78, ...,   0,   1,   1],
        [ 64,  67,  75, ...,   0,   1,   0],
        [ 65,  65,  74, ...,   0,   0,   0],
        ...,
        [138, 134, 138, ...,   2,   3,   3],
        [136, 132, 139, ...,   3,   3,   3],
        [135, 132, 139, ...,   5,   4,   4]],

       [[ 62,  66,  78, ...,   0,   1,   1],
        [ 64,  67,  75, ...,   0,   1,   0],
        [ 65,  65,  74, ...,   0,   0,   0],
        ...,
        [138, 134, 138, ...,   2,   3,   3],
        [136, 132, 139, ...,   3,   3,   3],
        [135, 132, 139, ...,   5,   4,   4]],

       [[ 62,  66,  78, ...,   0,   1,   1],
        [ 64,  67,  75, ...,   0,   1,   0],
        [ 65,  65,  74, ...,   0,   0,   0],
        ...,
        [138, 134, 138, ...,   2,   3,   3],
        [136, 132, 139, ...,   3,   3,   3],
        [135, 132, 139, ...,   5,   4,   4]]], dtype=uint8)

In [40]:
a = get_mask(image_dimensions[0:2], int(diagnostic.x_min), int(diagnostic.y_min), int(diagnostic.x_max), int(diagnostic.y_max))

In [41]:
img_mask = np.full((image_dimensions[0],image_dimensions[1]),0)
img_mask[1:10] = 255
img_mask

array([[  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [42]:
samples_df

,image_id,CategoryId,EncodedPixels,Rad_id,Height,Width
0,7ecd6f67f649f26c05805c8359f9e528,"[11, 13]",[169029 27 169541 27 170053 27 170565 27 17107...,R9,512,512
1,7ecd6f67f649f26c05805c8359f9e528,[13],[71265 62 71777 62 72289 62 72801 62 73313 62 ...,R10,512,512
2,7ecd6f67f649f26c05805c8359f9e528,[13],[77419 60 77931 60 78443 60 78955 60 79467 60 ...,R8,512,512
3,2229148faa205e881cf0d932755c9e40,"[13, 11, 7, 11]",[59464 61 59976 61 60488 61 61000 61 61512 61 ...,R9,512,512
4,2229148faa205e881cf0d932755c9e40,"[11, 13, 13, 13, 11]",[34887 99 35399 99 35911 99 36423 99 36935 99 ...,R8,512,512
...,...,...,...,...,...,...
13177,c619a784636c085eb798f98a5ba1102d,"[9, 0]",[41733 64 42245 64 42757 64 43269 64 43781 64 ...,R9,512,512
13178,c619a784636c085eb798f98a5ba1102d,"[9, 8, 0]",[43276 46 43788 46 44300 46 44812 46 45324 46 ...,R10,512,512
13179,e7ea94b2fec95a7461b10a7f3eea2897,"[0, 3]",[147546 57 148058 57 148570 57 149082 57 14959...,R10,512,512
13180,e7ea94b2fec95a7461b10a7f3eea2897,"[3, 0]",[116950 76 117462 76 117974 76 118486 76 11899...,R9,512,512
